Step 1 — Set up your Databricks workspace

In [0]:
CREATE CATALOG IF NOT EXISTS ecommerce_cohort;
CREATE SCHEMA IF NOT EXISTS ecommerce_cohort.bronze;
CREATE SCHEMA IF NOT EXISTS ecommerce_cohort.silver;
CREATE SCHEMA IF NOT EXISTS ecommerce_cohort.gold;


In [0]:
SHOW TABLES IN workspace.ecommerce_cohort

Step 2 — Load the 2024 dataset

In [0]:
CREATE OR REPLACE VIEW ecommerce_cohort.src_orders_2024 AS
SELECT
  row_id,
  customer_id,
  order_date,
  order_id,
  sales
FROM workspace.ecommerce_cohort.ecom_orders_2024_rebalanced;


In [0]:
SELECT
  COUNT(*) AS rows,
  MIN(order_date) AS min_date,
  MAX(order_date) AS max_date
FROM ecommerce_cohort.src_orders_2024;



Step 3 — Bronze layer: raw_orders

In [0]:
DESCRIBE TABLE src_orders_2024;


In [0]:
-- Create the Bronze Delta table

CREATE OR REPLACE TABLE ecommerce_cohort.bronze.raw_orders (
  row_id BIGINT,
  customer_id STRING,
  order_date DATE,
  order_id STRING,
  sales DOUBLE
)
USING DELTA;




In [0]:
-- Insert raw 2024 orders into the Bronze layer
-- Map original column names to standardized column names

INSERT OVERWRITE TABLE ecommerce_cohort.bronze.raw_orders
SELECT
  row_id,
  TRIM(customer_id) AS customer_id,
  order_date,
  TRIM(order_id) AS order_id,
  sales
FROM ecommerce_cohort.src_orders_2024;




In [0]:
-- Validate the Bronze table by checking volume, uniqueness, and date range

SELECT
  COUNT(*) AS row_count,
  COUNT(DISTINCT order_id) AS distinct_orders,
  COUNT(DISTINCT customer_id) AS customers,
  MIN(order_date) AS min_order_date,
  MAX(order_date) AS max_order_date
FROM ecommerce_cohort.bronze.raw_orders;




Step 4 — Silver layer: clean_orders

In [0]:
-- Create the Silver Delta table
-- This table stores clean and deduplicated order data
CREATE OR REPLACE TABLE ecommerce_cohort.silver.clean_orders (
  row_id BIGINT,
  customer_id STRING,
  order_date DATE,
  order_id STRING,
  sales DOUBLE
)
USING DELTA;



In [0]:
-- Insert validated and deduplicated orders into the Silver layer
-- Invalid rows are removed and only one row per order_id is kept
INSERT OVERWRITE TABLE ecommerce_cohort.silver.clean_orders
SELECT
  row_id,
  customer_id,
  order_date,
  order_id,
  sales
FROM (
  SELECT
    *,
    ROW_NUMBER() OVER (
      PARTITION BY order_id
      ORDER BY order_date
    ) AS rn
  FROM ecommerce_cohort.bronze.raw_orders
  WHERE order_id IS NOT NULL
    AND customer_id IS NOT NULL
    AND order_date IS NOT NULL
    AND sales IS NOT NULL
    AND sales > 0
)
WHERE rn = 1;



In [0]:
-- Validate that the number of rows matches the number of distinct order_id
SELECT
  COUNT(*) AS rows,
  COUNT(DISTINCT order_id) AS distinct_orders,
  COUNT(customer_id) AS customers,
  MIN(order_date) AS min_order_date,
  MAX(order_date) AS max_order_date
FROM ecommerce_cohort.silver.clean_orders;



In [0]:
show tables in ecommerce_cohort

In [0]:
SELECT
   * 
FROM ecommerce_cohort.silver.clean_orders
LIMIT 10;

Step 5 — Gold layer: cohort_customers

In [0]:
-- Create the Gold customer-level cohort table
CREATE OR REPLACE TABLE ecommerce_cohort.gold.cohort_customers (
  customer_id STRING,
  first_purchase_date DATE,
  second_purchase_date DATE,
  cohort_month DATE,
  days_to_second_purchase INT
)
USING DELTA;



In [0]:

INSERT OVERWRITE TABLE ecommerce_cohort.gold.cohort_customers

-- Step 1: Calculate the first purchase date per customer

WITH first_purchase AS (
  SELECT
    customer_id,
    MIN(order_date) AS first_purchase_date
  FROM ecommerce_cohort.silver.clean_orders
  GROUP BY customer_id
),

-- Step 2: Calculate the second purchase date per customer
-- Earliest order_date that occurs after the first purchase

second_purchase AS (
  SELECT
    o.customer_id,
    MIN(o.order_date) AS second_purchase_date
  FROM ecommerce_cohort.silver.clean_orders o
  JOIN first_purchase f
    ON o.customer_id = f.customer_id
   AND o.order_date > f.first_purchase_date
  GROUP BY o.customer_id
)
-- Step 3: Build the final cohort dataset
-- Add cohort month and days to second purchase

SELECT
  f.customer_id,
  f.first_purchase_date,
  s.second_purchase_date,
  DATE_TRUNC('month', f.first_purchase_date) AS cohort_month,
  DATEDIFF(s.second_purchase_date, f.first_purchase_date) AS days_to_second_purchase
FROM first_purchase f
LEFT JOIN second_purchase s
  ON f.customer_id = s.customer_id



In [0]:
--Validate total customers
SELECT COUNT(*) 
FROM ecommerce_cohort.gold.cohort_customers;



In [0]:
--Validate: repeaters vs one-time customers

SELECT
  CASE
    WHEN second_purchase_date IS NULL THEN 'one_time'
    ELSE 'repeater'
  END AS customer_type,
  COUNT(*) AS customers
FROM ecommerce_cohort.gold.cohort_customers
GROUP BY 1;



In [0]:
--Validate: sample rows look correct

SELECT *
FROM ecommerce_cohort.gold.cohort_customers
ORDER BY first_purchase_date
LIMIT 10;


Step 6 — Build reporting queries (cohort
metrics)

Step 7 — Dashboard for 2024 (3 charts)

In [0]:
-- Retention to 2nd Purchase by Cohort (1–3 months)
SELECT
  cohort_month,
  COUNT(*) AS cohort_size,

  SUM(
    CASE 
      WHEN second_purchase_date IS NOT NULL
       AND months_between(second_purchase_date, first_purchase_date) <= 1
      THEN 1 ELSE 0 
    END
  ) / COUNT(*) AS retention_1m,

  SUM(
    CASE 
      WHEN second_purchase_date IS NOT NULL
       AND months_between(second_purchase_date, first_purchase_date) <= 2
      THEN 1 ELSE 0 
    END
  ) / COUNT(*) AS retention_2m,

  SUM(
    CASE 
      WHEN second_purchase_date IS NOT NULL
       AND months_between(second_purchase_date, first_purchase_date) <= 3
      THEN 1 ELSE 0 
    END
  ) / COUNT(*) AS retention_3m

FROM ecommerce_cohort.gold.cohort_customers
GROUP BY cohort_month
ORDER BY cohort_month;


Databricks visualization. Run in Databricks to view.

In [0]:
-- Repeat Purchase Depth by Cohort

WITH orders_per_customer AS (
  SELECT
    customer_id,
    COUNT(*) AS total_orders
  FROM ecommerce_cohort.silver.clean_orders
  GROUP BY customer_id
)

SELECT
  c.cohort_month,

  SUM(CASE WHEN o.total_orders >= 2 THEN 1 ELSE 0 END) / COUNT(*) AS repeat_2plus,
  SUM(CASE WHEN o.total_orders >= 3 THEN 1 ELSE 0 END) / COUNT(*) AS repeat_3plus,
  SUM(CASE WHEN o.total_orders >= 4 THEN 1 ELSE 0 END) / COUNT(*) AS repeat_4plus

FROM ecommerce_cohort.gold.cohort_customers c
JOIN orders_per_customer o
  ON c.customer_id = o.customer_id

GROUP BY c.cohort_month
ORDER BY c.cohort_month;



Databricks visualization. Run in Databricks to view.

In [0]:
-- Cohort Size: number of new customers per cohort month
SELECT
  cohort_month,
  COUNT(*) AS new_customers
FROM ecommerce_cohort.gold.cohort_customers
GROUP BY cohort_month
ORDER BY cohort_month;


Databricks visualization. Run in Databricks to view.

Step 8 — Load the 2025 dataset (new
data arrives)

In [0]:
SHOW TABLES IN ecommerce_cohort;


In [0]:
-- Validate the 2025 source table
-- Check row count and ensure dates are only from 2025

SELECT
  COUNT(*) AS total_rows,
  MIN(order_date) AS min_order_date,
  MAX(order_date) AS max_order_date
FROM ecommerce_cohort.src_orders_2025;



Step 9 — Append 2025 data and rebuild Silver/Gold

In [0]:
-- Append 2025 orders into the existing Bronze table
-- This simulates the arrival of new data
INSERT INTO ecommerce_cohort.bronze.raw_orders
SELECT
  row_id,
  TRIM(customer_id) AS customer_id,
  order_date,
  TRIM(order_id) AS order_id,         
  sales
FROM ecommerce_cohort.src_orders_2025;


In [0]:
-- Validate Bronze after appending 2025 data

SELECT
  COUNT(*) AS total_rows,
  MIN(order_date) AS min_order_date,
  MAX(order_date) AS max_order_date
FROM ecommerce_cohort.bronze.raw_orders;


In [0]:
-- Rebuild the Silver layer using updated Bronze data (2024 + 2025)
-- Apply data quality rules and deduplicate orders
INSERT OVERWRITE TABLE ecommerce_cohort.silver.clean_orders
SELECT
  row_id,
  customer_id,
  order_date,
  order_id,
  sales
FROM (
  SELECT
    *,
    ROW_NUMBER() OVER (
      PARTITION BY order_id           
      ORDER BY order_date             
    ) AS rn
  FROM ecommerce_cohort.bronze.raw_orders
  WHERE order_id IS NOT NULL
    AND customer_id IS NOT NULL
    AND order_date IS NOT NULL
    AND sales IS NOT NULL
    AND sales > 0
)
WHERE rn = 1;                          


In [0]:
-- Validate Silver layer

SELECT
  COUNT(*) AS total_rows,
  COUNT(DISTINCT order_id) AS distinct_orders,
  MIN(order_date) AS min_order_date,
  MAX(order_date) AS max_order_date
FROM ecommerce_cohort.silver.clean_orders;


In [0]:
-- Rebuild the Gold cohort table using updated Silver data
-- Identify first and second purchases per customer

INSERT OVERWRITE TABLE ecommerce_cohort.gold.cohort_customers
WITH ordered_purchases AS (
  SELECT
    customer_id,
    order_date,
    ROW_NUMBER() OVER (
      PARTITION BY customer_id        
      ORDER BY order_date             
    ) AS purchase_rank
  FROM ecommerce_cohort.silver.clean_orders
),
first_second AS (
  SELECT
    customer_id,
    MIN(CASE WHEN purchase_rank = 1 THEN order_date END) AS first_purchase_date,
    MIN(CASE WHEN purchase_rank = 2 THEN order_date END) AS second_purchase_date
  FROM ordered_purchases
  GROUP BY customer_id
)
SELECT
  customer_id,
  first_purchase_date,
  second_purchase_date,
  DATE_TRUNC('month', first_purchase_date) AS cohort_month,
  DATEDIFF(second_purchase_date, first_purchase_date) AS days_to_second_purchase
FROM first_second;


In [0]:
-- Validate Gold layer
-- Check customer counts and presence of 2025 cohorts
SELECT
  COUNT(*) AS total_customers,
  COUNT(second_purchase_date) AS customers_with_second_purchase
FROM ecommerce_cohort.gold.cohort_customers;


In [0]:
-- Check cohort months
SELECT DISTINCT cohort_month
FROM ecommerce_cohort.gold.cohort_customers
ORDER BY cohort_month;
